In [1]:
import csv
from __future__ import division, unicode_literals

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [3]:
books = pd.read_json('Projects/4Fletcher/books_data.json')

In [4]:
books = books.reset_index()

In [5]:
books = books.drop('index',axis=1)

In [7]:
books.head()

,Author,Book_Title,Genre,Published_date,Summary
0,George Orwell,Animal Farm,"[Satire, Children's literature, Speculative fi...",1945,"Old Major, the old boar on the Manor Farm, ca..."
1,Anthony Burgess,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction...",1962,"Alex, a teenager living in near-future Englan..."
2,Robert Wilson,The Hidden Assassins,"[Crime Fiction, Mystery, Fiction]",2006,The novel tackles themes of terrorism.
3,Cecily von Ziegesar,Lucky,[Young adult literature],2007,Jenny Humphrey has attended some crazy partie...
4,Alexander McCall Smith,The Full Cupboard of Life,"[Mystery, Detective fiction]",2003,Mma Ramotswe is now engaged to the mechanic M...


In [8]:
books = books[books['Summary']!=None]

In [9]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12841 entries, 0 to 12840
Data columns (total 5 columns):
Author            12841 non-null object
Book_Title        12841 non-null object
Genre             12841 non-null object
Published_date    12841 non-null int64
Summary           12841 non-null object
dtypes: int64(1), object(4)
memory usage: 601.9+ KB


In [10]:
model = gensim.models.Word2Vec.load_word2vec_format('./Projects/4Fletcher/GoogleNews-vectors-negative300.bin', binary=True) 

In [11]:
import re, string
from string import maketrans
from gensim.models import Word2Vec

In [12]:
summ = [row for row in books['Summary']]

In [70]:
def tokenize(docs): 
    stoplist = stopwords.words('english')
    wordsindoc = []
    for doc in docs:
        words = []
        try:
            regex = re.compile('[%s]' % re.escape(string.punctuation))
            doc = doc.lower().split()
            for word in doc:
                if word not in stoplist:
                    try:
                        word = str(word)
                        word = regex.sub('',word)
                        words.append(word)
                    except:
                        words.append(np.NaN)
        except:
            words.append([])
        wordsindoc.append(words)
    return wordsindoc

In [71]:
def get_doc_vect(model,column_of_documents):
    wordtokens = tokenize(column_of_documents)
    doc_vects = []
    for words in wordtokens:
        if words:
            words_vecs = []
            for word in words:
                try:
                    words_vecs.append(model[word])
                except:
                    words_vecs.append(np.NaN)
            doc_vects.append(np.mean(words_vecs,axis=0))
        else:
            doc_vects.append(np.NaN)
    return doc_vects

In [73]:
testing = tokenize(summ[:5])

In [136]:
wordtokens = tokenize(summ[:5])
doc_v = []
for words in wordtokens:
    if words:
        words_vecs = []
        for word in words:
            try:
                words_vecs.append(model[word])
#                 words_vecs.append(model[word])
            except:
                words_vecs.append('**************')
        doc_v.append(words_vecs)
#                 words_vecs.append('')
#             try:
#                 words_vecs.append(model[word])
#             except:
#                 continue
# #                 words_vecs.append(np.NaN)
#         doc_v.append(words_vecs)
#         doc_vects.append(np.array(words_vecs).mean(axis=0))
#     else:
#         doc_v.append(np.NaN)

In [138]:
len(doc_v)

5

In [72]:
get_doc_vect(model,summ[:4])

[array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan, 

In [47]:
vects = get_doc_vect(model,summ)

ValueError: setting an array element with a sequence.

In [45]:
vects[:3]

[array([ 0.02722273,  0.0467241 ,  0.0097215 ,  0.08736935, -0.05503968,
         0.0194334 ,  0.06494179, -0.0298545 ,  0.03433893,  0.07145312,
        -0.00144809, -0.14629346, -0.05329417,  0.04213794, -0.12553762,
         0.05032607, -0.01903097,  0.1130823 , -0.02167779, -0.04025298,
         0.04126411, -0.01724179,  0.03971605,  0.00268001,  0.02105913,
        -0.04409397, -0.10374682,  0.0775576 ,  0.03552955, -0.04514285,
        -0.06199048, -0.03127785, -0.01873985,  0.03406183, -0.02926375,
         0.01857483,  0.02774816,  0.00169977,  0.059669  ,  0.0641543 ,
         0.09983871, -0.03587566,  0.10180657,  0.00270694, -0.01899396,
        -0.09912755, -0.00650425,  0.03710119, -0.01495402, -0.0010657 ,
        -0.0545496 ,  0.00339187,  0.0121142 , -0.007359  , -0.00572869,
         0.00340809, -0.01649153, -0.10736103,  0.01468519, -0.05637486,
         0.00458666,  0.08295171, -0.02107019, -0.07069165, -0.01175274,
        -0.04282688, -0.02875854,  0.07293853, -0.0

In [16]:
vects_array = np.array(vects)

In [19]:
len(vects)

33369097

In [17]:
vectors = pd.DataFrame(data=vects_array)

In [20]:
vectors.head()

,0
0,nan
1,nan
2,nan
3,None
4,nan


In [27]:
vectors.to_pickle('book_vectors.pkl')

In [28]:
cosine_similarity(vects_array)

AttributeError: 'DataFrame' object has no attribute 'fit'

In [18]:
books['Vectors'] = vects

ValueError: Length of values does not match length of index

In [34]:
books.head()

,Author,Book_Title,Genre,Published_date,Summary,Vectors
0,George Orwell,Animal Farm,"[Satire, Children's literature, Speculative fi...",1945,"Old Major, the old boar on the Manor Farm, ca...","[0.0272227, 0.0467241, 0.0097215, 0.0873694, -..."
1,Anthony Burgess,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction...",1962,"Alex, a teenager living in near-future Englan...",None
2,Robert Wilson,The Hidden Assassins,"[Crime Fiction, Mystery, Fiction]",2006,The novel tackles themes of terrorism.,"[0.124573, 0.105804, -0.0895081, 0.140381, -0...."
3,Cecily von Ziegesar,Lucky,[Young adult literature],2007,Jenny Humphrey has attended some crazy partie...,None
4,Alexander McCall Smith,The Full Cupboard of Life,"[Mystery, Detective fiction]",2003,Mma Ramotswe is now engaged to the mechanic M...,"[0.00286722, 0.0452107, 0.000550202, 0.0489288..."


In [43]:

count = 0
for row in books['Vectors']:
    if str(row) == 'None':
        count += 1

In [44]:
count

4008

In [51]:

stoplist = stopwords.words('english')


In [52]:
wordsdoc

[['alex'],
 [],
 ['teenager'],
 ['living'],
 [],
 ['nearfuture'],
 ['england'],
 ['leads'],
 [],
 ['gang'],
 [],
 ['nightly'],
 ['orgies'],
 [],
 ['opportunistic'],
 ['random'],
 ['ultraviolence'],
 ['alexs'],
 ['friends'],
 ['droogs'],
 [],
 [],
 ['novels'],
 ['anglorussian'],
 ['slang'],
 ['nadsat'],
 ['are'],
 ['dim'],
 [],
 ['slowwitted'],
 ['bruiser'],
 [],
 [],
 [],
 ['gangs'],
 ['muscle'],
 ['georgie'],
 [],
 ['ambitious'],
 ['secondincommand'],
 [],
 ['pete'],
 [],
 ['mostly'],
 ['plays'],
 ['along'],
 [],
 [],
 ['droogs'],
 ['indulge'],
 [],
 ['taste'],
 [],
 ['ultraviolence'],
 ['characterized'],
 [],
 [],
 ['sociopath'],
 [],
 [],
 ['hardened'],
 ['juvenile'],
 ['delinquent'],
 ['alex'],
 [],
 ['also'],
 ['intelligent'],
 [],
 ['quickwitted'],
 [],
 ['sophisticated'],
 ['taste'],
 [],
 ['music'],
 [],
 ['particularly'],
 ['fond'],
 [],
 ['beethoven'],
 [],
 ['lovely'],
 ['ludwig'],
 ['van'],
 [],
 ['novel'],
 ['begins'],
 [],
 [],
 ['droogs'],
 ['sitting'],
 [],
 [],
 ['favo